# Install

In [ ]:
#!pip install xgboost scikit-learn pandas numpy

# Imports

In [47]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Dataset

In [18]:
df = pd.read_csv('..\models\df_balanced.csv', low_memory=False)
display(df)

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,Grade 0,NaN,Unknown,20 G Infiltration Scale,Access Lines - Peripheral,16.350000,2724,DIRECT EMER.,Other,WHITE,CLINIC REFERRAL,67,11.4,11.4,g/dL,STAT,15.783333
1,Children,NaN,Unknown,Support Systems,Restraint/Support Systems,16.266667,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,32.9,32.9,pg,ROUTINE,0.500000
2,Unknown,NaN,Unknown,Restraint Location,Restraint/Support Systems,7.066667,2724,SURGICAL SAME DAY ADMISSION,Medicare,OTHER,PHYSICIAN REFERRAL,66,0,0.0,mEq/L,Unknown,4.000000
3,24,24.0,insp/min,Respiratory Rate,Respiratory,7.033333,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79,136,136.0,mEq/L,STAT,8.750000
4,Brisk,NaN,Unknown,Pupil Response Right,Neurological,21.550000,2724,URGENT,Other,WHITE,TRANSFER FROM HOSPITAL,56,110,110.0,IU/L,STAT,11.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,Oral,NaN,Unknown,Temperature Site,Routine Vital Signs,10.583333,Z87891,EW EMER.,Other,WHITE,EMERGENCY ROOM,70,178,178.0,mg/dL,STAT,3.850000
37196,11,11.0,mmHg,Central Venous Pressure,Hemodynamics,15.700000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,INTUBATED.,NaN,Unknown,Unknown,5.616667
37197,98,98.0,mmHg,Non Invasive Blood Pressure systolic,Routine Vital Signs,22.550000,Z87891,URGENT,Medicare,UNKNOWN,TRANSFER FROM HOSPITAL,81,12.5,12.5,%,ROUTINE,18.600000
37198,1,1.0,Unknown,Visual / hearing deficit,Adm History/FHPA,0.466667,Z87891,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,89,7.2,7.2,mg/dL,ROUTINE,10.083333


In [19]:
df.describe()

,valuenum_chartevent,time_since_admission_chartevent,age,valuenum_labevent,time_since_admission_labevent
count,16620.000000,37200.000000,37200.000000,32362.000000,37198.000000
mean,61.456809,12.032965,63.062715,100.226511,8.516042
std,553.498344,6.826768,14.245839,1281.318384,8.208554
min,-23.000000,-3.250000,29.000000,-21.000000,-20.300000
25%,1.700000,6.150000,58.000000,3.800000,3.366667
50%,19.000000,11.883333,63.000000,15.000000,8.750000
75%,83.000000,17.850000,72.000000,47.900000,14.266667
max,62656.000000,24.000000,91.000000,62656.000000,23.983333


In [20]:
df.describe(exclude=np.number)

,value_chartevent,valueuom_chartevent,label_chartevent,category,icd_code,admission_type,insurance,race,admission_location,value_labevent,valueuom_labevent,priority
count,37200,37200,37200,37200,37200,37200,37200,37200,37200,34214,37200,37200
unique,1734,31,784,25,6,6,3,9,7,1056,31,3
top,1,Unknown,Safety Measures,Routine Vital Signs,2724,EW EMER.,Other,WHITE,EMERGENCY ROOM,___,mEq/L,STAT
freq,1314,26705,1134,5509,6200,20023,18905,28009,15807,3150,6920,18274


# Treatment

## valuenum_chartevent -> Leave with the Nans

In [24]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
print(pd.DataFrame({"Missing": missing, "Percent (%)": missing_percent}))

                                 Missing  Percent (%)
value_chartevent                       0     0.000000
valuenum_chartevent                20580    55.322581
valueuom_chartevent                    0     0.000000
label_chartevent                       0     0.000000
category                               0     0.000000
time_since_admission_chartevent        0     0.000000
icd_code                               0     0.000000
admission_type                         0     0.000000
insurance                              0     0.000000
race                                   0     0.000000
admission_location                     0     0.000000
age                                    0     0.000000
value_labevent                      2986     8.026882
valuenum_labevent                      0     0.000000
valueuom_labevent                      0     0.000000
priority                               0     0.000000
time_since_admission_labevent          2     0.005376


In [25]:
# valuenum_labevent

numeric_cols = ['valuenum_labevent', 'age', 'time_since_admission_chartevent']

imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

display(df)

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,Grade 0,NaN,Unknown,20 G Infiltration Scale,Access Lines - Peripheral,16.350000,2724,DIRECT EMER.,Other,WHITE,CLINIC REFERRAL,67.0,11.4,11.40,g/dL,STAT,15.783333
1,Children,NaN,Unknown,Support Systems,Restraint/Support Systems,16.266667,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79.0,32.9,32.90,pg,ROUTINE,0.500000
2,Unknown,NaN,Unknown,Restraint Location,Restraint/Support Systems,7.066667,2724,SURGICAL SAME DAY ADMISSION,Medicare,OTHER,PHYSICIAN REFERRAL,66.0,0,0.00,mEq/L,Unknown,4.000000
3,24,24.0,insp/min,Respiratory Rate,Respiratory,7.033333,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79.0,136,136.00,mEq/L,STAT,8.750000
4,Brisk,NaN,Unknown,Pupil Response Right,Neurological,21.550000,2724,URGENT,Other,WHITE,TRANSFER FROM HOSPITAL,56.0,110,110.00,IU/L,STAT,11.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,Oral,NaN,Unknown,Temperature Site,Routine Vital Signs,10.583333,Z87891,EW EMER.,Other,WHITE,EMERGENCY ROOM,70.0,178,178.00,mg/dL,STAT,3.850000
37196,11,11.0,mmHg,Central Venous Pressure,Hemodynamics,15.700000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58.0,INTUBATED.,65.96,Unknown,Unknown,5.616667
37197,98,98.0,mmHg,Non Invasive Blood Pressure systolic,Routine Vital Signs,22.550000,Z87891,URGENT,Medicare,UNKNOWN,TRANSFER FROM HOSPITAL,81.0,12.5,12.50,%,ROUTINE,18.600000
37198,1,1.0,Unknown,Visual / hearing deficit,Adm History/FHPA,0.466667,Z87891,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,89.0,7.2,7.20,mg/dL,ROUTINE,10.083333


In [26]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
print(pd.DataFrame({"Missing": missing, "Percent (%)": missing_percent}))

                                 Missing  Percent (%)
value_chartevent                       0     0.000000
valuenum_chartevent                20580    55.322581
valueuom_chartevent                    0     0.000000
label_chartevent                       0     0.000000
category                               0     0.000000
time_since_admission_chartevent        0     0.000000
icd_code                               0     0.000000
admission_type                         0     0.000000
insurance                              0     0.000000
race                                   0     0.000000
admission_location                     0     0.000000
age                                    0     0.000000
value_labevent                      2986     8.026882
valuenum_labevent                      0     0.000000
valueuom_labevent                      0     0.000000
priority                               0     0.000000
time_since_admission_labevent          2     0.005376


In [27]:
df['value_labevent'] = df['value_labevent'].fillna('Unknown')


In [28]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
print(pd.DataFrame({"Missing": missing, "Percent (%)": missing_percent}))

                                 Missing  Percent (%)
value_chartevent                       0     0.000000
valuenum_chartevent                20580    55.322581
valueuom_chartevent                    0     0.000000
label_chartevent                       0     0.000000
category                               0     0.000000
time_since_admission_chartevent        0     0.000000
icd_code                               0     0.000000
admission_type                         0     0.000000
insurance                              0     0.000000
race                                   0     0.000000
admission_location                     0     0.000000
age                                    0     0.000000
value_labevent                         0     0.000000
valuenum_labevent                      0     0.000000
valueuom_labevent                      0     0.000000
priority                               0     0.000000
time_since_admission_labevent          2     0.005376


In [29]:
df_ = df.dropna(subset=['time_since_admission_labevent'])

missing = df_.isnull().sum()
missing_percent = (missing / len(df_)) * 100
print(pd.DataFrame({"Missing": missing, "Percent (%)": missing_percent}))

display(df_)

                                 Missing  Percent (%)
value_chartevent                       0     0.000000
valuenum_chartevent                20578    55.320179
valueuom_chartevent                    0     0.000000
label_chartevent                       0     0.000000
category                               0     0.000000
time_since_admission_chartevent        0     0.000000
icd_code                               0     0.000000
admission_type                         0     0.000000
insurance                              0     0.000000
race                                   0     0.000000
admission_location                     0     0.000000
age                                    0     0.000000
value_labevent                         0     0.000000
valuenum_labevent                      0     0.000000
valueuom_labevent                      0     0.000000
priority                               0     0.000000
time_since_admission_labevent          0     0.000000


,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,Grade 0,NaN,Unknown,20 G Infiltration Scale,Access Lines - Peripheral,16.350000,2724,DIRECT EMER.,Other,WHITE,CLINIC REFERRAL,67.0,11.4,11.40,g/dL,STAT,15.783333
1,Children,NaN,Unknown,Support Systems,Restraint/Support Systems,16.266667,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79.0,32.9,32.90,pg,ROUTINE,0.500000
2,Unknown,NaN,Unknown,Restraint Location,Restraint/Support Systems,7.066667,2724,SURGICAL SAME DAY ADMISSION,Medicare,OTHER,PHYSICIAN REFERRAL,66.0,0,0.00,mEq/L,Unknown,4.000000
3,24,24.0,insp/min,Respiratory Rate,Respiratory,7.033333,2724,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,79.0,136,136.00,mEq/L,STAT,8.750000
4,Brisk,NaN,Unknown,Pupil Response Right,Neurological,21.550000,2724,URGENT,Other,WHITE,TRANSFER FROM HOSPITAL,56.0,110,110.00,IU/L,STAT,11.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,Oral,NaN,Unknown,Temperature Site,Routine Vital Signs,10.583333,Z87891,EW EMER.,Other,WHITE,EMERGENCY ROOM,70.0,178,178.00,mg/dL,STAT,3.850000
37196,11,11.0,mmHg,Central Venous Pressure,Hemodynamics,15.700000,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58.0,INTUBATED.,65.96,Unknown,Unknown,5.616667
37197,98,98.0,mmHg,Non Invasive Blood Pressure systolic,Routine Vital Signs,22.550000,Z87891,URGENT,Medicare,UNKNOWN,TRANSFER FROM HOSPITAL,81.0,12.5,12.50,%,ROUTINE,18.600000
37198,1,1.0,Unknown,Visual / hearing deficit,Adm History/FHPA,0.466667,Z87891,EW EMER.,Medicare,WHITE,EMERGENCY ROOM,89.0,7.2,7.20,mg/dL,ROUTINE,10.083333


# XGBoost DMatrix

In [45]:
# Extract feature and target arrays
X = df_.drop('icd_code', axis=1)
y = df_['icd_code']

In [48]:

le = LabelEncoder()
y_encoded = le.fit_transform(y)

y = y_encoded

In [49]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [50]:
print(X.dtypes)

value_chartevent                   category
valuenum_chartevent                 float64
valueuom_chartevent                category
label_chartevent                   category
category                           category
time_since_admission_chartevent     float64
admission_type                     category
insurance                          category
race                               category
admission_location                 category
age                                 float64
value_labevent                     category
valuenum_labevent                   float64
valueuom_labevent                  category
priority                           category
time_since_admission_labevent       float64
dtype: object


In [51]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Model

In [55]:
params = {
    'max_depth': 3,
    'eta': 1,
    'objective': 'multi:softprob',  # para classificação multi-classe
    'num_class': len(le.classes_),  # número de doenças diferentes
    'eval_metric': 'merror',      # métrica de avaliação or mlogloss
    'tree_method': 'hist',          # método rápido e eficiente
    'seed': 42                      # para reprodutibilidade
}

In [60]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

In [65]:
preds = model.predict(dtest)
y_pred = preds.argmax(axis=1)

from sklearn.metrics import accuracy_score, classification_report

print("Acuraccy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


Acuraccy: 0.6134408602150537
              precision    recall  f1-score   support

        2724       0.70      0.71      0.70      1268
        4019       0.70      0.69      0.70      1271
        E039       0.59      0.57      0.58      1243
        E785       0.43      0.37      0.40      1229
        Z794       0.73      0.85      0.79      1190
      Z87891       0.48      0.49      0.49      1239

    accuracy                           0.61      7440
   macro avg       0.61      0.61      0.61      7440
weighted avg       0.61      0.61      0.61      7440

